Celem zadania była implementacja algorytmu ID3 oraz przebadanie jego działania na zbiorze danych Titanic https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv


In [ ]:
#author Michał Iskra 

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split



#Mikro testy

Boxes = {'Color': ['Green','Green','Green','Blue','Blue','Red','Red','Red'],
         'Shape': ['Rectangle','Rectangle','Square','Rectangle','Square','Square','Square','Rectangle'],
         'Class': [1,1,1,0,1,0,1,0]
        }
df = pd.DataFrame(Boxes, columns= ['Color','Shape', 'Class'])
X = df[['Color','Shape','Class']]





#TITANIC Dane

data = pd.read_csv('titanic.csv')

#usunięcie kolumny z imionami
data_final = data.drop(columns = ['Name'])

#zamiana wartości ceny biletu na wartości dyskretne
data_final['Fare'] = data_final['Fare'].map(lambda a: round(a,-1) )

#podział na podzbiory treningowy i testowy
train, test = train_test_split(data_final, test_size=0.2)

#usunięcie kolumny klasy której szukamy
test_features = test.drop(columns = ['Survived'])



#Klasa opisująca węzeł
class Node:
  def __init__(self,data,depth):
    self.data = data
    self.depth = depth
    self.classifier = None
    self.children = {}
    self.best_column = None
  

#Algorytm
class ID3:

  def __init__(self, dataset, class_name, max_depth):
    self.dataset = dataset                               #dane do treningu
    self.class_name = class_name                         #nazwa kolumny z klasami które chcemy badać
    self.rootNode = Node(self.dataset, depth = 0)        #Stworzenie pierwszego węzła
    self.max_depth = max_depth                           #maksymalna głębokość
    self.generateNodes(self.rootNode,self.rootNode.data) #funkcja do stworzenia drzewa
    
    

  def best_feature(self,data):

    #entropia aktualnego zestawu danych
    current_entropy = self.calc_entropy(data[self.class_name])

    #wydzielenie kolumn z atrybutami 
    features = data.loc[:, data.columns != self.class_name]

    attributes_dict = {} 
    
    for column in features:

      unique_values, counts = np.unique(features[column], return_counts=True) #znalezienie różnych atrybutów i ich ilości
      sum = np.sum(counts)                                                    #wyliczenie ilości do wyliczenia prawdopodobieństwa
    
      attributes_dict[column] = 0.0                                           #ważona entropia dla danej kolumny

      for i in range(len(unique_values)):

        probability_i = counts[i]/sum
        entropy_i = self.calc_entropy(data.loc[data[column]==unique_values[i],self.class_name]) 
        attributes_dict[column] += probability_i * entropy_i
      
    max_gain = float('-inf')                  
    result = list(attributes_dict.keys())[0]
   
    for key in attributes_dict:                     #szukanie największego przyrostu informacji

      gain = current_entropy - attributes_dict[key]

      if gain > max_gain:
        max_gain = gain
        result = key

    return key
    
      
      

  def calc_entropy(self,data): #funkcja do wyliczania entropii z kolumny danych

    unique_values, counts = np.unique(data, return_counts=True)

    entropy_list = []

    for i in range(len(unique_values)):
      prob = counts[i]/np.sum(counts) 
      entropy_list.append(-prob*np.log2(prob))

    return np.sum(entropy_list)



  def generateNodes(self, root, data):


    
    unique_values, counts = np.unique(data[self.class_name], return_counts=True)

    #sprawdzenie czy nie została już tylko jedna klasa
    if len(unique_values) == 1: 
      root.classifier = data[self.class_name].to_numpy()[0]

      

    #sprawdzenie czy zostały jakiekolwiek atrybuty lub czy jest maks głębokość,
    #przypisanie klasyfikatora do rodzica jako klasa której zostało najwięcej

    elif len(data.columns) <= 1 or root.depth == self.max_depth:
      root.classifier = np.argmax(np.unique(data[self.class_name], return_counts=True)[1])

      
    #znalezienine najlepszej kolumny atrybutu według której można podzielić
    #oraz wyznaczenie nazw, wartości atrybutów względem których dzielę
    else:

      best_column = self.best_feature(data)
      root.best_column = best_column


      unique_values = np.unique(data[best_column])

      for item in unique_values:

        data_temp = data[data[best_column]==item]
        data_out = data_temp.drop(columns=[best_column])

        root.children[item] = Node(data_out, depth = root.depth+1)


      for child_key in root.children:
       
        child = root.children[child_key]
        self.generateNodes(child, child.data)
        

  def predict(self, features, root=None):
    

    if root == None:
      root = self.rootNode

    if root.classifier is None:   #jeżeli węzeł rodzic nie ma klasyfikatora to znaczy że ma dzieci
      column = root.best_column

      for key in root.children.keys():
        if features[column] == key:
          return self.predict(features, root.children[key])
    else:                        #jeżeli ma to dotarliśmy do liścia
      return root.classifier
      
    
    


drzewo = ID3(X,'Class',3)
wynik = drzewo.predict(features = y)
#print(wynik)

tree = ID3(data_final,'Survived',3) #mniejsza głębokość

#testy na wejście test_features, potem porównuję z test

print("Survived or not ------- >{}".format(tree.predict(test_features.iloc[0])))
print(test.iloc[0])
print("\n")

print("Survived or not  ------->  {}".format(tree.predict(test_features.iloc[1])))
print(test.iloc[1])
print("\n")

print("Survived or not ------->  {}".format(tree.predict(test_features.iloc[2])))
print(test.iloc[2])
print("\n")

tree2 = ID3(data_final,'Survived',10) #większa glębokość

print("Survived or not ------- >{}".format(tree2.predict(test_features.iloc[0])))
print(test.iloc[0])
print("\n")

print("Survived or not  ------->  {}".format(tree2.predict(test_features.iloc[1])))
print(test.iloc[1])
print("\n")

print("Survived or not ------->  {}".format(tree2.predict(test_features.iloc[2])))
print(test.iloc[2])
print("\n")





Survived or not ------- >0
Survived                        1
Pclass                          2
Sex                        female
Age                            29
Siblings/Spouses Aboard         0
Parents/Children Aboard         0
Fare                           10
Name: 65, dtype: object


Survived or not  ------->  0
Survived                      0
Pclass                        2
Sex                        male
Age                          46
Siblings/Spouses Aboard       0
Parents/Children Aboard       0
Fare                         30
Name: 395, dtype: object


Survived or not ------->  0
Survived                        0
Pclass                          3
Sex                        female
Age                            25
Siblings/Spouses Aboard         1
Parents/Children Aboard         0
Fare                           10
Name: 725, dtype: object


Survived or not ------- >1
Survived                        1
Pclass                          2
Sex                        female
Age    